## Cilia quantification
The purpose of this notebook is to implement scikit image regionprops to populate a table with the characteristics of each segmented cilia.

In [ ]:
import matplotlib.pyplot as plt
import napari
import numpy as np
import pandas as pd
from readlif.reader import LifFile
from skimage.exposure import rescale_intensity
from skimage.io import imread
from skimage.measure import regionprops_table
import seaborn as sns
from morphocilia.io import lifloader

In [ ]:
img = lifloader(
    "D:/estela/data/microscopy/leica_stellaris/20231025_p1_cd13_opn_arl13b/20231025_p1_cd13_opn_arl13b.lif",
    5,
)
cilia_channel = img[1]
labelled_prediction = imread("D:/estela/src/labelled_prediction.tif")
classes_for_training = imread("D:/estela/src/classes_for_training.tif")

viewer = napari.Viewer()
viewer.add_image(cilia_channel)
viewer.add_labels(labelled_prediction, opacity=1)
viewer.add_labels(classes_for_training, opacity=1)
napari.utils.nbscreenshot(viewer)

In [ ]:
def intensity_median(mask, intensity_image):
    return np.median(intensity_image[mask])

In [ ]:
props = regionprops_table(
    labelled_prediction,
    cilia_channel,
    properties=[
        "label",
        "area",
        "axis_major_length",
        "axis_minor_length",
        "intensity_max",
        "solidity",
    ],
    extra_properties=[intensity_median],
)
data = pd.DataFrame(props)
data

In [ ]:
def parse_classes(class_number):
    if class_number == 1:
        return "elongated"
    elif class_number == 2:
        return "looped"
    elif class_number == 3:
        return "fibroblastic"
    elif class_number == 0:
        return "NA"
    else:
        raise ValueError

In [ ]:
props = regionprops_table(
    labelled_prediction,
    classes_for_training,
    properties=["label", "intensity_max"],
)
data_classes = pd.DataFrame(props).rename(columns={"intensity_max": "classes"})
data_classes["classes"] = data_classes["classes"].apply(parse_classes)
data_classes

In [ ]:
complete_dataset = data.merge(right=data_classes, on="label")
complete_dataset

In [ ]:
sns.pairplot(data)

In [ ]:
sns.pairplot(
    complete_dataset.query("classes != 'NA'").query("area < 25000"),
    hue="classes",
    plot_kws={"alpha": 0.4},
)

In [ ]:
complete_dataset.to_csv("cilia_quantification.csv", index=False)

# Tests

In [ ]:
complete_dataset.query("axis_major_length > 60")

In [ ]:
complete_dataset.loc[complete_dataset["area"].argmax()]